# IMPROVED MODEL WEBCAM DEPLOYMENT

In [15]:
import time
import tensorflow as tf
import numpy as np
from PIL import Image
from keras import models
from keras.models import model_from_json
from facenet_pytorch import MTCNN
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device='cuda:0')
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv


mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


video = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0


model_name = "model_main2_WORKING"
FPS_CAP=10




# load json and create model
json_file = open(model_name+'/model.json', 'r') 
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(model_name+"/model.h5")
print("Loaded model from disk")

class_names = {0 : 'BORED', 1: 'CONFUSED', 2 : 'DELIGHT', 3 : 'FRUSTRATED', 4 : 'NEUTRAL'}
C_FPS = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5,model_complexity =0) as holistic:
        
        while True:
                _, frame = video.read()
                new_frame_time = time.time()
                fps = 1/(new_frame_time-prev_frame_time)
                prev_frame_time = new_frame_time
                fps = int(fps)
                fps = str(fps)
                C_FPS = C_FPS +1

                #Convert the captured frame into RGB
                im = Image.fromarray(frame, 'RGB')
                
                #Resizing into 160x160 because we trained the model with this image size.
                #im = im.resize((160,160))
                 
                # Recolor Feed
                image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image1.flags.writeable = False        
                # Recolor image back to BGR for rendering
                image1.flags.writeable = True   
                image = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
                if C_FPS == FPS_CAP:
                        faces = mtcnn(im)

                        try:
                                for im in faces:
                                        # DL MODEL
                                        im = im.permute(1,2,0)
                                        img_array = np.expand_dims(im, axis=0)
                                        prediction = model.predict(img_array,verbose = 0)
                                        maxindex = int(np.argmax(prediction))
                                        prediction = class_names[maxindex]

                                        # Make Detections
                                        results = holistic.process(image1)
                                        # print(results.face_landmarks)

                        except:
                                prediction = "NONE"
                        C_FPS = 0


                try:
                        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
                        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                                )

                        # Left Hand
                        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                                )

                        # Pose Detections
                        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                                )
                        pose = results.pose_landmarks.landmark
                        h1 = results.right_hand_landmarks.landmark
                        h2 = results.left_hand_landmarks.landmark

                except:
                        _


                try:
                        if (pose[7].x-pose[3].x) < 0.01:
                                #cv2.putText(image, (str("BORED"+ "{:.2f}".format((pose[7].x-pose[3].x)))), (5, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                                prediction = "BORED"
                        elif(pose[6].x-pose[8].x) < 0.01:
                                #cv2.putText(image, (str("BORED"+ "{:.2f}".format((pose[6].x-pose[8].x)))), (5, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                                prediction = "BORED"       
                        else:
                                #cv2.putText(image, (str("Not Distracted"+ "{:.2f}".format((pose[7].x-pose[3].x)))), (5, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                                _ 
                        cv2.putText(image, str(prediction), (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

                except:
                        _


                cv2.putText(image, ("FPS:"+fps), (5, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.imshow("DETECTION", image)
                key=cv2.waitKey(1)
                if key == ord('q'):
                        break
video.release()
cv2.destroyAllWindows()

Loaded model from disk
